<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Sentiment analysis

Veremos dos modelos o implementaciones distintas:

- Basado en diccionario
- Basado en métodos de aprendizaje automático (machine learning, ML)

El conjunto de datos sobre el que trabajaremos está disponible en: 
https://www.kaggle.com/welkin10/airline-sentiment

Estos son datos de aerolíneas de EE. UU. que contienen comentarios de pasajeros sobre la base del servicio proporcionado por las aerolíneas, donde nos interesa poder clasificar los comentarios según sean positivos o negativos.

El conjunto de datos contiene:

- 14.640 comentarios etiquetados
- 15 atributos, incluyendo la polaridad del comentario

## Pre-procesamiento de los datos

En primer lugar, y antes de realizar cualquier tipo de análisis, es común (y muy útil) realizar una serie de procesos sobre el texto para mejorar los análisis posteriores. 

En concreto, estos procesos se pueden dividir en 3 bloques:

- Tokenización del texto
- Eliminación de las *stopwords*
- Stemming (obtención de la palabra raíz)

In [2]:
import pandas as pd
import numpy as np

dataset = pd.read_csv("https://raw.githubusercontent.com/kolaveridi/kaggle-Twitter-US-Airline-Sentiment-/master/Tweets.csv", encoding="ISO-8859-1")

print("Dataset shape is {}".format(dataset.shape))
dataset.head()

Dataset shape is (14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


Deberemos instalar la librería "nltk" de procesamiento de lenguaje natural, y descargar el conjunto de datos eqtiquetado como "stopwords".

In [3]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Cargamos los atributos "text" (que contiene el texto del tweet) y "airline_sentiment" (que contiene la polaridad del tweet).

Además, cargamos:

- las *stopwords* (en inglés) 
- y la clase que realizará el *stemmer* (que es un proceso automatizado que produce una cadena de base en un intento de representar palabras relacionadas)

In [0]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

tweets = dataset.text
results = dataset.airline_sentiment
english_stopwords = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

In [5]:
for i in range(5):
    print("[%d] [%s\t]: %s" % (i, results[i], tweets[i]))

[0] [neutral	]: @VirginAmerica What @dhepburn said.
[1] [positive	]: @VirginAmerica plus you've added commercials to the experience... tacky.
[2] [neutral	]: @VirginAmerica I didn't today... Must mean I need to take another trip!
[3] [negative	]: @VirginAmerica it's really aggressive to blast obnoxious "entertainment" in your guests' faces &amp; they have little recourse
[4] [negative	]: @VirginAmerica and it's a really big bad thing about it


In [6]:
print("Stop words [num=%d] and sample is %s" % ((len(english_stopwords), list(english_stopwords)[0:5])))

Stop words [num=179] and sample is ["you'd", 'between', 'about', 'is', "you've"]


### Step 1 - Tokenize

La primera técnica a aplicar es la tokenización, la cual consiste en separar un texto por palabras, sentencias o conjunto de palabras. En este caso, como que los tweets son frases cortas, se ha separado por palabras.

In [0]:
def word_tokenizing(txts):
    regexT = RegexpTokenizer('[a-zA-Z\']+')
    txts_tokenized = []
    
    for txt in txts:
        a = regexT.tokenize(txt)
        txts_tokenized.append(a)
    return(txts_tokenized)

tweets_tokenized = word_tokenizing(tweets)

In [9]:
for i in range(5):
    print("[%d] -> %s\n    <- %s" % (i, tweets_tokenized[i], tweets[i]))

[0] -> ['VirginAmerica', 'What', 'dhepburn', 'said']
    <- @VirginAmerica What @dhepburn said.
[1] -> ['VirginAmerica', 'plus', "you've", 'added', 'commercials', 'to', 'the', 'experience', 'tacky']
    <- @VirginAmerica plus you've added commercials to the experience... tacky.
[2] -> ['VirginAmerica', 'I', "didn't", 'today', 'Must', 'mean', 'I', 'need', 'to', 'take', 'another', 'trip']
    <- @VirginAmerica I didn't today... Must mean I need to take another trip!
[3] -> ['VirginAmerica', "it's", 'really', 'aggressive', 'to', 'blast', 'obnoxious', 'entertainment', 'in', 'your', "guests'", 'faces', 'amp', 'they', 'have', 'little', 'recourse']
    <- @VirginAmerica it's really aggressive to blast obnoxious "entertainment" in your guests' faces &amp; they have little recourse
[4] -> ['VirginAmerica', 'and', "it's", 'a', 'really', 'big', 'bad', 'thing', 'about', 'it']
    <- @VirginAmerica and it's a really big bad thing about it


### Step 2 - Remove StopWords

Una vez tenemos el conjunto de palabras por separado, hay que eliminar las llamadas *stopwords*, que son palabras muy frecuentes pero no tienen ninguna información semántica, como por ejemplo los determinantes y adverbios.

In [0]:
def stopword_removal(txts):
    t_s = []
    tweets_stopword = []
    
    for txt in txts:
        for word in txt:
            if word not in english_stopwords:
                t_s.append(word)
        tweets_stopword.append(t_s)
        t_s = []
    return(tweets_stopword)

tweets_stopword = stopword_removal(tweets_tokenized)

In [11]:
for i in range(5):
    print("[%d] -> %s\n    <- %s" % (i, tweets_stopword[i], tweets_tokenized[i]))

[0] -> ['VirginAmerica', 'What', 'dhepburn', 'said']
    <- ['VirginAmerica', 'What', 'dhepburn', 'said']
[1] -> ['VirginAmerica', 'plus', 'added', 'commercials', 'experience', 'tacky']
    <- ['VirginAmerica', 'plus', "you've", 'added', 'commercials', 'to', 'the', 'experience', 'tacky']
[2] -> ['VirginAmerica', 'I', 'today', 'Must', 'mean', 'I', 'need', 'take', 'another', 'trip']
    <- ['VirginAmerica', 'I', "didn't", 'today', 'Must', 'mean', 'I', 'need', 'to', 'take', 'another', 'trip']
[3] -> ['VirginAmerica', 'really', 'aggressive', 'blast', 'obnoxious', 'entertainment', "guests'", 'faces', 'amp', 'little', 'recourse']
    <- ['VirginAmerica', "it's", 'really', 'aggressive', 'to', 'blast', 'obnoxious', 'entertainment', 'in', 'your', "guests'", 'faces', 'amp', 'they', 'have', 'little', 'recourse']
[4] -> ['VirginAmerica', 'really', 'big', 'bad', 'thing']
    <- ['VirginAmerica', 'and', "it's", 'a', 'really', 'big', 'bad', 'thing', 'about', 'it']


### Step 3 - Stemming

La técnica conocida como *stemming* es el proceso de transformar las palabras a su forma raíz. De esta forma mantenemos la semántica y unificamos todas las palabras derivadas a una única palabra (la raíz).

In [0]:
def stemming(txts):
    tweets_stemmed = []
    
    for txt in txts:
        stem = []
        for word in txt:
            stem.append(stemmer.stem(word))
        tweets_stemmed.append(stem)
    
    return(tweets_stemmed)

tweets_stemmed = stemming(tweets_stopword)

In [13]:
for i in range(5):
    print("[%d] -> %s\n    <- %s" % (i, tweets_stemmed[i], tweets_stopword[i]))

[0] -> ['virginamerica', 'what', 'dhepburn', 'said']
    <- ['VirginAmerica', 'What', 'dhepburn', 'said']
[1] -> ['virginamerica', 'plus', 'ad', 'commerci', 'experi', 'tacki']
    <- ['VirginAmerica', 'plus', 'added', 'commercials', 'experience', 'tacky']
[2] -> ['virginamerica', 'i', 'today', 'must', 'mean', 'i', 'need', 'take', 'anoth', 'trip']
    <- ['VirginAmerica', 'I', 'today', 'Must', 'mean', 'I', 'need', 'take', 'another', 'trip']
[3] -> ['virginamerica', 'realli', 'aggress', 'blast', 'obnoxi', 'entertain', 'guest', 'face', 'amp', 'littl', 'recours']
    <- ['VirginAmerica', 'really', 'aggressive', 'blast', 'obnoxious', 'entertainment', "guests'", 'faces', 'amp', 'little', 'recourse']
[4] -> ['virginamerica', 'realli', 'big', 'bad', 'thing']
    <- ['VirginAmerica', 'really', 'big', 'bad', 'thing']


## Sentiment Analysis

Veremos dos opciones muy comunes:

- Uso de dictionarios
- Word2Vec + aprendizaje automático (Machine Learning, ML)

### Opción 1: Sentiment Analysis using dictionaries

En este primer caso emplearemos diccionarios para evaluar la polaridad (positivo / negativo) de las palabras que aparecen en el texto, de forma independiente.

Existen multitud de diccionarios disponibles en Internet. En este caso hemos empleado los diccionarios disponibles en: 
https://www.kaggle.com/andyxie/sentiment-analysis-dictionary

In [14]:
# carga de los diccionarios
positive_words = pd.read_csv("https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/positive-words.txt", encoding="ISO-8859-1", comment=";")
negative_words = pd.read_csv("https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/negative-words.txt", encoding="ISO-8859-1", comment=";")

positive_words = np.resize(positive_words[:].values, len(positive_words))
negative_words = np.resize(negative_words[:].values, len(negative_words))

print("El diccionario positivo está formado por {} palabras".format(len(positive_words)))
print(positive_words)
print("El diccionario negativo está formado por {} palabras".format(len(negative_words)))
print(negative_words)

print("abound" in positive_words)
print("abound" in negative_words)

El diccionario positivo está formado por 2005 palabras
['abound' 'abounds' 'abundance' ... 'zenith' 'zest' 'zippy']
El diccionario negativo está formado por 4782 palabras
['2-faces' 'abnormal' 'abolish' ... 'zealous' 'zealously' 'zombie']
True
False


In [20]:
# función para obtener el score de cada texto
def sentiment_analysis(txts):
    predicted = []

    for txt in txts:
        m = 0
        for word in txt:
            if word in positive_words:
                m = m+1
            elif word in negative_words:
                m = m-1
        
        if m > 0:
            label = "positive"
        elif m < 0:
            label = "negative"
        else:
            label = "neutral"
        predicted.append(label)
        
    return(predicted)

predicted = sentiment_analysis(tweets_stemmed)

print("El resumen de la predicción es: {} positivos, {} negativos y {} neutros"
  .format(predicted.count('positive'), predicted.count('negative'), predicted.count('neutral')))

El resumen de la predicción es: 3963 positivos, 3819 negativos y 6858 neutros


In [22]:
for i in range(5):
    print("[%d] : Predcited: %s \t-> Real: %s" % (i, predicted[i], results[i]))

[0] : Predcited: neutral 	-> Real: neutral
[1] : Predcited: neutral 	-> Real: positive
[2] : Predcited: neutral 	-> Real: neutral
[3] : Predcited: positive 	-> Real: negative
[4] : Predcited: negative 	-> Real: negative


A partir del número de aciertos sobre el número total de textos se define la precisión (*accuracy*) del modelo.

In [23]:
ok = sum(predicted == results)
total = len(results)

print("Accuracy is %.2f%% [%d / %d]" % ((ok / total)*100, ok, total))

Accuracy is 47.53% [6959 / 14640]


### Opción 2: Word2Vec + modelo ML

En este segundo caso emplearemos un método un poco más complejo. 

Este método consiste en dos pasos principales:

- Crear una matriz deonde cada fila (vector) contenga la información de un tweet
- Crear y entrenar un modelos de aprendizaje automático (ML) que sea capaz de clasificar nuevos textos

#### Crear un vector con todas las palabras pre-procesadas

En concreto, crearemos un vector que nos permite obtener la siguiente información:

- Crear un vector donde las columnas representan cada una (TODAS) de las palabras del vocabulario (es decir, de TODOS los textos que emplearemos)
- Crear una nueva fila en este vector para cada uno de los textos (tweets), donde un 0 indica que la palabra representada en la columna NO aparece en el texto, y 1 que la palabra sí aparece.

In [0]:
tweets_stemmed_joined = []

for tweet in tweets_stemmed:
    tweets_stemmed_joined.append(" ".join(str(x) for x in tweet))

In [25]:
for i in range(5):
    print("[%d] -> %s\n    <- %s" % (i, tweets_stemmed_joined[i], tweets_stemmed[i]))

[0] -> virginamerica what dhepburn said
    <- ['virginamerica', 'what', 'dhepburn', 'said']
[1] -> virginamerica plus ad commerci experi tacki
    <- ['virginamerica', 'plus', 'ad', 'commerci', 'experi', 'tacki']
[2] -> virginamerica i today must mean i need take anoth trip
    <- ['virginamerica', 'i', 'today', 'must', 'mean', 'i', 'need', 'take', 'anoth', 'trip']
[3] -> virginamerica realli aggress blast obnoxi entertain guest face amp littl recours
    <- ['virginamerica', 'realli', 'aggress', 'blast', 'obnoxi', 'entertain', 'guest', 'face', 'amp', 'littl', 'recours']
[4] -> virginamerica realli big bad thing
    <- ['virginamerica', 'realli', 'big', 'bad', 'thing']


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized_corpus = vectorizer.fit_transform(tweets_stemmed_joined)

In [27]:
vectorized_names = vectorizer.get_feature_names()
print("El vocabulario tiene una longitud de {} palabras, y un ejemplo es {}"
      .format(len(vectorized_names), vectorized_names[0:10]))

print("La matriz tiene un tamaño de {}".format(vectorized_corpus.shape))

for i in range(5):
    print("[%d] -> %s\n%s" % (i, tweets_stemmed_joined[i], vectorized_corpus[i,:]))

El vocabulario tiene una longitud de 10898 palabras, y un ejemplo es ['aa', 'aaaand', 'aaadvantag', 'aaalwaysl', 'aacustomerservic', 'aadavantag', 'aadelay', 'aadv', 'aadvantag', 'aafail']
La matriz tiene un tamaño de (14640, 10898)
[0] -> virginamerica what dhepburn said
  (0, 10107)	1
  (0, 10337)	1
  (0, 2319)	1
  (0, 8070)	1
[1] -> virginamerica plus ad commerci experi tacki
  (0, 10107)	1
  (0, 7178)	1
  (0, 89)	1
  (0, 1749)	1
  (0, 3015)	1
  (0, 9065)	1
[2] -> virginamerica i today must mean i need take anoth trip
  (0, 10107)	1
  (0, 9418)	1
  (0, 6148)	1
  (0, 5783)	1
  (0, 6250)	1
  (0, 9075)	1
  (0, 385)	1
  (0, 9544)	1
[3] -> virginamerica realli aggress blast obnoxi entertain guest face amp littl recours
  (0, 10107)	1
  (0, 7637)	1
  (0, 152)	1
  (0, 945)	1
  (0, 6597)	1
  (0, 2846)	1
  (0, 3900)	1
  (0, 3057)	1
  (0, 330)	1
  (0, 5408)	1
  (0, 7672)	1
[4] -> virginamerica realli big bad thing
  (0, 10107)	1
  (0, 7637)	1
  (0, 893)	1
  (0, 672)	1
  (0, 9278)	1


#### Crear y entrenar un modelo de ML

En este caso emplearemos un modelo (relativamente) simple y que ofrece (generalmente) unos buenos resultados.

Las *Suport Vector Machine* o SVM son un conjunto de algoritmos de aprendizaje supervisado utilizados en problemas de clasificación y regresión. 

A partir de un conjunto de ejemplos de entrenamiento (etiquetados) podemos entrenar una SVM para construir un modelo que prediga la etiqueta o clase de una nueva muestra. Intuitivamente, una SVM es un modelo que representa a los puntos de la muestra en un espacio de N dimensiones, donde N es el número de atributos. El objetivo es separar las clases en espacios lo más amplios posibles mediante un hiperplano de separación.

Más información en: https://en.wikipedia.org/wiki/Support-vector_machine

El primer paso antes de entrenar un modelo basado en aprendizaje supervisado es partir los datos en dos conjuntos disjuntos:

- Conjunto de entrenamiento (*train*)
- Conjunto de test (*test*)

El conjunto de entrenamiento se utilizará para optimizar (entrenar) el modelo, mientras que el conjunto de test nos permitirá "medir" la precisión del modelo al predecir nuevos datos.

Muchas veces se crean empleando la regla del 80-20, es decir, 80% de instancias para entrenar y 20% restante para realizar el test.

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(vectorized_corpus, results
                                                    , test_size=0.2
                                                    , random_state=42)

In [29]:
print("Train dataset shape is {}".format(X_train.shape))
print("Test dataset shape is  {}".format(X_test.shape))

Train dataset shape is (11712, 10898)
Test dataset shape is  (2928, 10898)


Una vez tenemos los conjuntos de datos creados, pasaremos a probar diferentes valores para los parámetros del modelo. 

En el caso de una SVM, los principales parémtros son:

- *Kernel*: función que se aplica a los datos para cambiar el espacio de representación
- *Gamma*: De manera intuitiva, el parámetro gamma define hasta dónde llega la influencia de un solo ejemplo de entrenamiento
- *C*: El parámetro C le dice a la optimización de SVM cuánto quiere evitar errores en la clasificación de cada ejemplo de entrenamiento

In [30]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

param_grid = {'C':[1,10,100]
              ,'gamma':[1,0.1,0.001]
              ,'kernel':['linear','rbf']}

grid_svm = GridSearchCV(svm.SVC(), param_grid, n_jobs=-1, refit=True, verbose=True)
grid_svm.fit(X_train,y_train)

print("Los mejores parámetros para la SVM y datos actuales son:")
print(grid_svm.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 23.8min finished


Los mejores parámetros para la SVM y datos actuales son:
{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}


Una vez hemos determinado los parámetros óptimos para el modelo y el conjunto de datos, debemos crear un nuevo modelo con estos parámetros y entrenarlo con el conjunto de datos correspondiente.

In [32]:
clf = svm.SVC(C=100, gamma=0.001, kernel='rbf',verbose=True)

# mdoel training
clf.fit(X_train, y_train)

# train and test evaluation
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

[LibSVM]

Finalmente, emplearemos el conjunto de test para obtener el valor de precisión (*accuracy*) conseguido por el modelo.

In [33]:
print("Train accuracy is {}%".format(np.mean(y_train_pred == y_train)*100))
print("Test accuracy is  {}%".format(np.mean(y_test_pred == y_test)*100))

Train accuracy is 88.82342896174863%
Test accuracy is  79.40573770491804%
